In [1]:
import wandb
import torch, wandb, os
import torch.optim as optim

from src.RandmanFunctions import RandmanConfig, split_and_load
from src.Models import RandmanSNN, spike_regularized_cross_entropy
from src.EvolutionAlgorithms.EvolutionStrategy import ESModel
from src.EvolutionAlgorithms.PseudoPSO import PPSOModel, PPSOModelWithPooling
from src.SurrogateGD.VanillaSGD import SGDModel
from src.Training import train_loop_snn
from src.Utilities import init_result_csv, set_seed
from src.LandscapeAnalysis.LossSurfacePlotter import LossSurfacePlotter

device = 'cuda'
loss_fn_spk = lambda logits, y, spikes: spike_regularized_cross_entropy(logits, y, spikes, lambda_reg=1e-3)

@torch.no_grad()
def train_snn():
    run_name = "wenqi-pipeline-test"
    config = {  # Dataset:
        "nb_input": 10,
        "nb_output": 10,
        "nb_steps": 50,
        "nb_data_samples": 1000,
        "dim_manifold": 2,
        "alpha": 2.0,
        # SNN:
        "nb_hidden": 100,
        "learn_beta": False,
        # Evolution Strategy:
        "nb_model_samples": 100,
        "mirror": True,
        # Training:
        "std": 0.1,
        "epochs": 5,
        "batch_size": 256,
        "method": "es",
        # Optimization:
        "loss": "loss_fn",
        "optimizer": "Adam",
        "lr": 0.01,
        "regularization": "none",
    }
    with wandb.init(
        entity="DarwinNeuron", project="Test", name=run_name, config=config
    ) as run:
        # update current run_name
        keys = ["method", "std", "batch_size", "lr", "nb_model_samples", "loss", "nb_input"]
        sorted_items = [f"{getattr(run.config, k)}" for k in sorted(keys)]
        run.name = "-".join(sorted_items)
        
        # setting up local csv recording (optional)
        # result_path, _, _ = init_result_csv(dict(run.config), run.project)

        my_model = ESModel(
            RandmanSNN,
            run.config.nb_input,
            run.config.nb_hidden,
            run.config.nb_output,
            run.config.learn_beta, 
            sample_size=run.config.nb_model_samples,
            param_std=run.config.std,
            Optimizer=optim.Adam,
            lr=run.config.lr,
            device=device,
            mirror=run.config.mirror,
        )

        # load dataset
        train_loader, val_loader = split_and_load(
            RandmanConfig(
                nb_classes=run.config.nb_output,
                nb_units=run.config.nb_input,
                nb_steps=run.config.nb_steps,
                nb_samples=run.config.nb_data_samples,
                dim_manifold=run.config.dim_manifold,
                alpha=run.config.alpha,
            ).read_dataset(),
            run.config.batch_size,
        )
        
        # loss surface plotter
        plotter_dir = f"results/{run.project}/runs/{run.id}/"
        os.makedirs(plotter_dir, exist_ok=True)
        loss_plotter = LossSurfacePlotter(plotter_dir+"illuminated_loss_surface.npz")

        # epochs
        for epoch in range(run.config.epochs):
            print(f"Epoch {epoch}\n-------------------------------")

            # train the model
            train_loop_snn(my_model, train_loader, val_loader, loss_fn_spk, device, run, epoch, loss_plotter=None)

train_snn()

wandb: Currently logged in as: yixing to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


filepath data/randman\a818832ebb0748ed8125b520de22a658.pt
Epoch 0
-------------------------------
batch 0, loss: 2.302585, accuracy: 13.7%
Accuracy: 10.2%, Avg loss: 2.302501 

batch 1, loss: 2.301536, accuracy: 12.5%
Accuracy: 10.2%, Avg loss: 2.302482 

batch 2, loss: 2.302965, accuracy: 10.5%
Accuracy: 10.2%, Avg loss: 2.302313 

batch 3, loss: 2.302818, accuracy: 10.9%
Accuracy: 10.3%, Avg loss: 2.302066 

batch 4, loss: 2.301008, accuracy: 10.2%
Accuracy: 10.4%, Avg loss: 2.301812 

batch 5, loss: 2.301578, accuracy: 9.0%
Accuracy: 10.5%, Avg loss: 2.301308 

batch 6, loss: 2.302917, accuracy: 10.2%
Accuracy: 10.7%, Avg loss: 2.300964 

batch 7, loss: 2.301794, accuracy: 11.7%
Accuracy: 10.9%, Avg loss: 2.300626 

batch 8, loss: 2.298819, accuracy: 10.5%
Accuracy: 11.2%, Avg loss: 2.299693 

batch 9, loss: 2.298710, accuracy: 11.7%
Accuracy: 11.5%, Avg loss: 2.298812 

batch 10, loss: 2.294088, accuracy: 11.3%
Accuracy: 11.8%, Avg loss: 2.297087 

batch 11, loss: 2.295508, accurac

Traceback (most recent call last):
  File "C:\Users\w1886\AppData\Local\Temp\ipykernel_25996\1159165513.py", line 92, in train_snn
    train_loop_snn(my_model, train_loader, val_loader, loss_fn_spk, device, run, epoch, loss_plotter=None)
  File "c:\Users\w1886\OneDrive\Documents\GitHub\DarwinNeuron\src\Training.py", line 43, in train_loop_snn
    my_model.update(get_model_stats)
  File "c:\Users\w1886\OneDrive\Documents\GitHub\DarwinNeuron\src\EvolutionAlgorithms\EvolutionStrategy.py", line 135, in update
    samples_loss.append(model_stats_fn(model).loss)
            ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\w1886\OneDrive\Documents\GitHub\DarwinNeuron\src\Training.py", line 41, in get_model_stats
    stats = run_snn_on_batch(model, x, y, loss_fn)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\w1886\OneDrive\Documents\GitHub\DarwinNeuron\src\Utilities.py", line 138, in run_snn_on_batch
    spikes, voltages = model(x)
                       ^^^^^^^^
  File "c:\Users\w

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅████
epoch_val_acc,▁█
epoch_val_average_neuron_spikes,▁█
epoch_val_loss,█▁
epoch_val_spike_percentage,▁█
train_acc,▃▃▁▂▁▁▂▁▂▂▇▅▅▃▂▁▂▁▄▄▅▅▄▃▅▆▆▇▄▄▄█▄▆▆▆▅▆▅▆
train_average_neuron_spikes,▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▄▃▄▄▅▅▆▆▇▆█▇▇██
train_loss,███████▇▇▇▆▅▇▆▅▇▆▇▅▆▅▄▆▅▄▄▄▅▄▄▁▅▄▂▂▁▂▂▃▂
train_spike_percentage,▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▄▄▅▅▆▆▆▆▇▇██▇
val_acc,▁▁▁▁▁▁▂▂▂▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▆▆▆▇▇█████
val_average_neuron_spikes,▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▄▄▄▅▅▆▆▆▇▇▇████


KeyboardInterrupt: 

In [2]:
import wandb
run = wandb.init(
    entity="DarwinNeuron",
    project="Test",
    name="wandb-connection-diag",
    config={"diag": True},
)

wandb: Currently logged in as: yixing to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


KeyboardInterrupt: 